**TDA Bird Songs Analysis**

This notebook analyzes bird songs using topological data analysis. To run this notebook in Google Colab, click the badge below:

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/navyanthkusampudi/TDA_BirdSongs/blob/main/Bird_Song_analysis.ipynb)

**Setup**

In [ ]:
#The code below will clone the repository and set up the environment in Google Colab.

import sys
import os

if 'google.colab' in sys.modules:
    # clone the repository only if running in google colab
    !git clone https://github.com/navyanthkusampudi/TDA_BirdSongs.git
    %cd TDA_BirdSongs
    
# Ensure the modules directory is in the Python path
#sys.path.append(os.path.abspath('TDA_BirdSongs/modules'))

**Import libraries**

In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np

# Add the directory containing the 'modules' package to the system path
sys.path.append(os.path.abspath('..'))


In [ ]:
from modules import utilityFunctions

Incase any libraby is not installed we recommend to do it using !pip install library --user

## Sonuds

#### Get data

In [ ]:
from modules.utilityFunctions import wav, wavPlayer

In [ ]:
#! wget https://github.com/navyRUB/nonlinear_DYnamics/raw/master/sound/data/XC629908_Great_Tit_Parus_major.wav
file = 'data/bird_sounds/XC629908_Great_Tit_Parus_major.wav'

In [ ]:
rate,data = wav.read(file)
print(data.shape)

In [ ]:
wavPlayer(file)

####  Select data range

In [ ]:
total_index = data.shape[0]
total_time = 0*60+16  #sec

from_time = 0*60 + 1#sec
to_time =0*60 +4#sec

# Extract data
from_index = np.int32(total_index*from_time/total_time)
to_index = np.int32(total_index*to_time/total_time)
try:
  sound = data[from_index:to_index,0]
except:
  sound = data[from_index:to_index]
sound.shape,rate

#### 1. Spectogram

In [ ]:
#@title Spectogram { display-mode: "form" }
# time series
plt.figure(figsize=(20,5),facecolor='white')
plt.rcParams['axes.facecolor']='white'
plt.specgram(sound, Fs=rate,cmap='viridis')
plt.xlabel('Time [sec]',color='black',fontsize=18)
plt.ylabel('Frequency [Hz]',color='black',fontsize=18)
plt.xticks(color='black',fontsize=20)
plt.yticks(color='black',fontsize=18)
plt.ylim(0,17500)
plt.show()

#### 2. Time series

In [ ]:
#@title Time seriest { display-mode: "form" }
# time series
plt.figure(figsize=(20,5),facecolor='white')
plt.rcParams['axes.facecolor']='white'
plt.plot(sound,'blue',alpha=0.7)
plt.xlabel('Data index',color='black',fontsize=20)
plt.ylabel('Amplitude',color='black',fontsize=20)
plt.xticks(color='black',fontsize=20)
plt.yticks(color='black',fontsize=20)
plt.xlim(0,90000)
plt.show()

#### 3. Sliders for Attractor

In [ ]:
from modules.utilityFunctions import parameter
from modules.utilityFunctions import plot_scatter, plot_attractor
from modules.utilityFunctions import widget_dynamics

In [ ]:
plot_scatter(sound)

In [ ]:
timeseries_data = sound
my_from_idx= 14770.00
my_size = 15726 - my_from_idx

In [ ]:
#@title Attractor { display-mode: "form" }
# plot Attractor


my_attractor_parameters = [parameter("index",my_from_idx,0,timeseries_data.shape[0]-1,300),
                           parameter("size",my_size,20,50000,50),
                           parameter("tau",3,1,300,1),
                           parameter("g_alpha",0.1,0.01,1.0,0.01),
                           parameter("g_size",1.0,0.01,10,0.01),
                           parameter("c_size",0.20,0.01,10,0.01)
                          ]
aa = utilityFunctions.widget_dynamics(function=plot_attractor,
               parameters=my_attractor_parameters,
               fixed_params=timeseries_data)

#### 4. Animation for Attractor

In [ ]:
from modules.utilityFunctions import get_delay_coordinates
from modules.utilityFunctions import plot_attractor_animation

In [ ]:
# animation
crop_size = 200
timeseries_data = sound
my_from_idx= 14770
my_size = 24770 - my_from_idx
x1,x2 = get_delay_coordinates(sound[my_from_idx:my_from_idx+my_size],2)
plot_attractor_animation(x1,x2,crop_size, sound)

#### Mutual information

In [ ]:
from modules.utilityFunctions import plot_mutual_information

In [ ]:
plot_mutual_information(sound[10000:10900])

## Topological data analysis

In [ ]:
from modules.utilityFunctions import delay_coordinates_highDim
from modules.utilityFunctions import PCA

In [ ]:
from ripser import ripser
from persim import plot_diagrams

In [ ]:
sound_selected = sound[24364:29957]
sound_selected.shape

In [ ]:
def TDA(zoom_start,window_size,tau,m):

  # data to plot
  y = sound_selected
  x = np.arange(0,len(y),1)
  # Define zoomed interval: i.e the data interval to select for analysis
  zoom_start = np.int32(zoom_start)
  window_size = np.int32(window_size)

  tau = np.int32(tau)
  m = np.int32(m)


  zoom_end = zoom_start + window_size


  # delay embedding
  embedding  = delay_coordinates_highDim(y[zoom_start:zoom_end], tau, m)

  # PCA
  pca = PCA(n_components=3)
  pca_embedding = pca.fit_transform(embedding)

  # persistent homology on PCA embeding
  if len(embedding)<1000:
    dgms = ripser(embedding,coeff=2,maxdim=1)['dgms']

  else:   #subsampling the manifold to save computation cost
    dgms = ripser(embedding,coeff=2,maxdim=1,n_perm=1000)['dgms']


  ################### Plotting



  # Create figure and subplots
  fig = plt.figure(figsize=(12,1))
  #ax = fig.add_subplot(131, projection='3d')

  # Plot full signal in ax1
  ax1 = fig.add_subplot(121)
  ax1.plot(x, y, 'y-', linewidth=1,alpha=0.2)
  ax1.scatter(x,y,c='y',s=2)
  ax1.set_ylabel('Signal')
  ax1.set_xlim([x[0], x[-1]])
  # Draw rectangle for zoomed interval in ax1
  rect = plt.Rectangle((zoom_start, np.min(y)), zoom_end - zoom_start, 2*np.max(y), facecolor='blue', alpha=0.5)
  ax1.add_patch(rect)

  # Plot zoomed interval in ax2
  ax2 = fig.add_subplot(122)
  ax2.plot(x[zoom_start:zoom_end], y[zoom_start:zoom_end], 'y-', linewidth=1,alpha=0.2)
  ax2.scatter(x[zoom_start:zoom_end], y[zoom_start:zoom_end], c='y',s=2)
  ax2.set_ylabel('Signal')
  ax2.set_xlim([zoom_start, zoom_end])
  plt.show()

  fig = plt.figure(figsize=(12,4))
  # pca
  ax = fig.add_subplot(121, projection='3d')
  ax.scatter(pca_embedding[:,0], pca_embedding[:,1], pca_embedding[:,2],
              c= np.linspace(0,1,len(pca_embedding)))
  #ax.set_aspect('equal')
  plt.title('PCA')

  # holology
  plt.subplot(122)
  plot_diagrams(dgms,size=20)
  plt.show()


In [ ]:
import matplotlib.pyplot as plt

plt.rcParams['text.usetex'] = False

In [ ]:
# define your parameters
my_parameters = [parameter("zoom_start",250,0,len(sound_selected),50),
                  parameter("window_size",1900,2,len(sound_selected)-200,10),
                 parameter("tau",3,1,400,1),
                 parameter("m",20,2,100,1)]

# call widgets
widget_dynamics(function=TDA,
               parameters=my_parameters)